<h1>Serve Pose Detection with CSV output and Wrist Rotation</h1>
<h2>Cell 1: Gets all the libraries needed</h2>
<h2>Cell 2: Code with csv output without bounding boxes but with arc of racquet</h2>

Cell 1

In [1]:
nvidia-smi

NameError: name 'nvidia' is not defined

In [1]:
import cv2
import mediapipe as mp
import math
import numpy as np
import torch
import cv2
import mediapipe as mp
import math
import numpy as np
from ultralytics import YOLO

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


Cell 2

In [ ]:
import Functions

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Try to import YOLO models with error handling
yolo_model_racquet = None
yolo_model_ball = None

try:
    from ultralytics import YOLO
    try:
        yolo_model_racquet = YOLO("D:\\DDSA\\YOLO_Models\\yolo11l_best.pt")
        yolo_model_racquet.to('cpu')
    except:
        print("YOLO racquet model not found, using simple detection")
    
    try:
        yolo_model_ball = YOLO("D:\\DDSA\\Serve Pose Detection\\ball_detection_model.pt")
        yolo_model_ball.to('cpu')
    except:
        print("YOLO ball model not found, using simple detection")
        
except ImportError:
    print("Ultralytics YOLO not available, using simple object detection methods")

# Base paths
input_video_dir = 'Your Input Directory'
output_video_dir = 'Your Output Directory'
os.makedirs(output_video_dir, exist_ok=True)

# Video files to process
video_files = ['1.mp4','2.mp4', '3.mp4', '4.mp4', '5.mp4']

# Reference object properties
reference_object_height_px = 100
reference_object_height_cm = 150
conversion_factor = reference_object_height_cm / reference_object_height_px

for video_file in video_files:
    file_name = os.path.splitext(video_file)[0]
    input_video_path = os.path.join(input_video_dir, video_file)
    
    # Open the video file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_file}")
        continue

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Output video setup
    output_video_path = os.path.join(output_video_dir, f'{file_name}_output_video.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Create CSV file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = os.path.join(output_video_dir, f'{file_name}_analysis_{timestamp}.csv')

    # Initialize variables for this video
    prev_wrist_position = None
    prev_elbow_position = None
    ball_release_time = None
    racquet_contact_time = None
    service_motion_time = None
    current_ball_height_cm = 0
    max_ball_height_cm = 0
    ball_ankle_height_cm = 0
    
    # Wrist arc tracking variables
    wrist_positions = []
    max_wrist_positions = 100
    serving_phase = False
    arc_curvature = 0
    max_arc_curvature = 0
    
    # Wrist rotation tracking variables
    wrist_angle_differences = []
    max_angular_velocity = 0
    current_angular_velocity = 0
    wrist_rotation_angle = 0

    with open(csv_filename, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        
        # Write CSV header
        header = ['frame_number', 'timestamp_ms']
        
        # Pose landmarks
        for landmark in mp_pose.PoseLandmark:
            header.extend([f'{landmark.name}_x', f'{landmark.name}_y', f'{landmark.name}_z', f'{landmark.name}_visibility'])
        
        # Ball and racquet positions
        header.extend(['ball_x', 'ball_y', 'ball_confidence', 'racquet_x', 'racquet_y', 'racquet_confidence'])
        
        # Calculated metrics
        header.extend([
            'upper_body_tilt', 'arm_angle', 'elbow_flexion', 'wrist_flexion', 
            'left_knee_flexion', 'right_knee_flexion', 'current_ball_height_cm',
            'max_ball_height_cm', 'ball_ankle_height_cm', 'ball_projection_angle',
            'lower_body_tilt', 'trunk_tilt', 'arm_speed', 'service_motion_time',
            'ball_release_time', 'racquet_contact_time', 'wrist_arc_curvature',
            'wrist_rotation_angle', 'wrist_angular_velocity'
        ])
        
        csv_writer.writerow(header)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
            
            # Initialize row data
            row_data = [frame_number, timestamp_ms]
            landmark_data = [None] * (33 * 4)
            ball_data = [None, None, None]
            racquet_data = [None, None, None]
            calculated_metrics = [None] * 19
            
            # Process frame
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results_pose = pose.process(image_rgb)

            # Process pose landmarks
            if results_pose.pose_landmarks:
                for i, landmark in enumerate(results_pose.pose_landmarks.landmark):
                    landmark_data[i*4] = landmark.x * frame_width
                    landmark_data[i*4+1] = landmark.y * frame_height
                    landmark_data[i*4+2] = landmark.z
                    landmark_data[i*4+3] = landmark.visibility
                
                mp_drawing.draw_landmarks(frame, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                # Get keypoints
                landmarks = results_pose.pose_landmarks.landmark
                shoulder_left = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
                shoulder_right = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                elbow_left = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
                elbow_right = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
                wrist_left = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
                wrist_right = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
                hip_left = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
                hip_right = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
                knee_left = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
                knee_right = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
                ankle_left = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
                ankle_right = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]

                # Calculate wrist and elbow positions for tracking
                wrist_x = wrist_right.x * frame_width
                wrist_y = wrist_right.y * frame_height
                current_wrist_pos = (wrist_x, wrist_y)
                
                elbow_x = elbow_right.x * frame_width
                elbow_y = elbow_right.y * frame_height
                current_elbow_pos = (elbow_x, elbow_y)

                # Detect serving phase (when wrist starts moving upward)
                if prev_wrist_position is not None:
                    if wrist_y < prev_wrist_position[1] and not serving_phase:
                        serving_phase = True
                        wrist_positions = []
                        print("Serve motion detected!")
                    
                    if serving_phase:
                        wrist_positions.append(current_wrist_pos)
                        if len(wrist_positions) > max_wrist_positions:
                            wrist_positions.pop(0)
                        
                        # Calculate arc curvature
                        if len(wrist_positions) >= 3:
                            arc_curvature = calculate_arc_curvature(wrist_positions)
                            max_arc_curvature = max(max_arc_curvature, arc_curvature)

                # Calculate wrist angular rotation
                if prev_wrist_position is not None and prev_elbow_position is not None:
                    angle_diff = calculate_wrist_rotation(prev_wrist_position, current_wrist_pos, 
                                                        prev_elbow_position, current_elbow_pos)
                    
                    wrist_rotation_angle += angle_diff
                    
                    # Store recent angle differences for angular velocity calculation
                    wrist_angle_differences.append(angle_diff)
                    if len(wrist_angle_differences) > 5:
                        wrist_angle_differences.pop(0)
                    
                    # Calculate angular velocity
                    current_angular_velocity = calculate_angular_velocity(wrist_angle_differences, fps)
                    max_angular_velocity = max(max_angular_velocity, abs(current_angular_velocity))

                # Draw wrist arc
                if len(wrist_positions) >= 2:
                    for i in range(1, len(wrist_positions)):
                        cv2.line(frame, 
                                (int(wrist_positions[i-1][0]), int(wrist_positions[i-1][1])),
                                (int(wrist_positions[i][0]), int(wrist_positions[i][1])),
                                (0, 255, 255), 3)

                # Draw current wrist position
                cv2.circle(frame, (int(wrist_x), int(wrist_y)), 10, (0, 0, 255), -1)

                # Calculate metrics
                upper_body_tilt = calculate_tilt(shoulder_left, shoulder_right)
                arm_angle = calculate_angle(shoulder_right, elbow_right, wrist_right)
                elbow_flexion = calculate_angle(shoulder_right, elbow_right, wrist_right)
                wrist_flexion = calculate_angle(elbow_right, wrist_right, (wrist_right.x + 0.1, wrist_right.y))
                
                if prev_wrist_position is not None:
                    arm_speed = calculate_speed(prev_wrist_position, current_wrist_pos, fps)
                    calculated_metrics[12] = arm_speed

                # Detect ball release and racquet contact based on wrist movement
                if prev_wrist_position is not None:
                    current_wrist_y = wrist_right.y * frame_height
                    
                    # Ball release: when wrist starts moving upward (y decreases)
                    if current_wrist_y < prev_wrist_position[1]:
                        if ball_release_time is None:
                            ball_release_time = timestamp_ms / 1000
                            calculated_metrics[14] = ball_release_time
                    
                    # Racquet contact: when wrist starts moving downward after release (y increases)
                    if ball_release_time is not None and current_wrist_y > prev_wrist_position[1]:
                        if racquet_contact_time is None:
                            racquet_contact_time = timestamp_ms / 1000
                            calculated_metrics[15] = racquet_contact_time
                            serving_phase = False

                # Update previous positions
                prev_wrist_position = current_wrist_pos
                prev_elbow_position = current_elbow_pos

                left_knee_flexion = calculate_angle(hip_left, knee_left, ankle_left)
                right_knee_flexion = calculate_angle(hip_right, knee_right, ankle_right)
                
                # Calculate ball height based on wrist position
                current_ball_height_cm = (frame_height - wrist_y) * conversion_factor
                if current_ball_height_cm > max_ball_height_cm:
                    max_ball_height_cm = current_ball_height_cm

                if ball_release_time is not None and racquet_contact_time is not None:
                    service_motion_time = racquet_contact_time - ball_release_time
                    calculated_metrics[13] = service_motion_time

                ball_projection_angle = calculate_angle(shoulder_right, wrist_right, (wrist_right.x + 0.1, wrist_right.y))
                lower_body_tilt = calculate_tilt(hip_left, hip_right)
                
                shoulder_midpoint = ((shoulder_left.x + shoulder_right.x) / 2, (shoulder_left.y + shoulder_right.y) / 2)
                hip_midpoint = ((hip_left.x + hip_right.x) / 2, (hip_left.y + hip_right.y) / 2)
                trunk_tilt = calculate_tilt(shoulder_midpoint, hip_midpoint)

                # Calculated metrics
                calculated_metrics[0] = upper_body_tilt
                calculated_metrics[1] = arm_angle
                calculated_metrics[2] = elbow_flexion
                calculated_metrics[3] = wrist_flexion
                calculated_metrics[4] = left_knee_flexion
                calculated_metrics[5] = right_knee_flexion
                calculated_metrics[6] = current_ball_height_cm
                calculated_metrics[7] = max_ball_height_cm
                calculated_metrics[8] = ball_ankle_height_cm
                calculated_metrics[9] = ball_projection_angle
                calculated_metrics[10] = lower_body_tilt
                calculated_metrics[11] = trunk_tilt
                calculated_metrics[16] = arc_curvature
                calculated_metrics[17] = wrist_rotation_angle
                calculated_metrics[18] = current_angular_velocity

                # Display results on frame
                y_offset = 30
                cv2.putText(frame, f"Wrist Arc Curvature: {arc_curvature:.1f} deg", (10, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                y_offset += 30
                cv2.putText(frame, f"Wrist Rotation: {wrist_rotation_angle:.1f} deg", (10, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                y_offset += 30
                cv2.putText(frame, f"Angular Velocity: {current_angular_velocity:.1f} deg/s", (10, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                y_offset += 30
                #cv2.putText(frame, f"Arm Speed: {arm_speed:.1f} px/s" if prev_wrist_position is not None else "Arm Speed: -", 
                #           (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                #y_offset += 25
                cv2.putText(frame, f"Ball Height: {current_ball_height_cm:.1f} cm", (10, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Add visual indicator for wrist rotation direction
                rotation_indicator_x = frame_width - 100
                rotation_indicator_y = 100
                rotation_radius = 30

                # Draw rotation indicator circle
                cv2.circle(frame, (rotation_indicator_x, rotation_indicator_y), rotation_radius, (255, 255, 255), 2)

                # Draw rotation direction arrow
                if abs(current_angular_velocity) > 10:
                    arrow_length = 25
                    if current_angular_velocity > 0:
                        # Clockwise rotation
                        cv2.arrowedLine(frame, 
                                       (rotation_indicator_x, rotation_indicator_y - arrow_length),
                                       (rotation_indicator_x, rotation_indicator_y + arrow_length),
                                       (0, 255, 0), 3)
                        cv2.putText(frame, "CW", (rotation_indicator_x - 15, rotation_indicator_y - 40),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    else:
                        # Counter-clockwise rotation
                        cv2.arrowedLine(frame, 
                                       (rotation_indicator_x, rotation_indicator_y + arrow_length),
                                       (rotation_indicator_x, rotation_indicator_y - arrow_length),
                                       (0, 0, 255), 3)
                        cv2.putText(frame, "CCW", (rotation_indicator_x - 20, rotation_indicator_y - 40),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

                # Add text label for rotation indicator
                cv2.putText(frame, "Wrist Rotation", (rotation_indicator_x - 50, rotation_indicator_y + 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

                # Draw elbow-wrist line to visualize the rotation
                cv2.line(frame, 
                        (int(elbow_x), int(elbow_y)),
                        (int(wrist_x), int(wrist_y)),
                        (255, 255, 0), 2)

                # Draw angle arc between consecutive elbow-wrist vectors
                if len(wrist_positions) >= 2 and len(wrist_angle_differences) > 0:
                    current_angle = math.degrees(math.atan2(wrist_y - elbow_y, wrist_x - elbow_x))
                    start_angle = current_angle - 30
                    end_angle = current_angle + 30
                    cv2.ellipse(frame, (int(wrist_x), int(wrist_y)), (20, 20), 0, 
                                start_angle, end_angle, (255, 100, 0), 2)

            # Process ball detection
            ball_detections = None
            if yolo_model_ball:
                try:
                    results_yolo_ball = yolo_model_ball(frame, conf=0.1)
                    if results_yolo_ball:
                        for result in results_yolo_ball:
                            boxes = result.boxes
                            for box in boxes:
                                if box.cls == 0:
                                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                                    ball_data = [(x1 + x2) / 2, (y1 + y2) / 2, box.conf.cpu().numpy()[0]]
                                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                except:
                    ball_detections = detect_ball_simple(frame)
            else:
                ball_detections = detect_ball_simple(frame)
            
            if ball_detections:
                for x, y, conf in ball_detections:
                    cv2.circle(frame, (int(x), int(y)), 15, (0, 255, 0), 2)
                    ball_data = [x, y, conf]

            # Process racquet detection
            racquet_detections = None
            if yolo_model_racquet:
                try:
                    results_yolo_racquet = yolo_model_racquet(frame, conf=0.1)
                    if results_yolo_racquet:
                        for result in results_yolo_racquet:
                            boxes = result.boxes
                            for box in boxes:
                                if box.cls == 0:
                                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                                    racquet_data = [(x1 + x2) / 2, (y1 + y2) / 2, box.conf.cpu().numpy()[0]]
                                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                except:
                    racquet_detections = detect_racquet_simple(frame)
            else:
                racquet_detections = detect_racquet_simple(frame)
            
            if racquet_detections:
                for x, y, conf in racquet_detections:
                    cv2.circle(frame, (int(x), int(y)), 20, (255, 0, 0), 2)
                    racquet_data = [x, y, conf]

            # Combine all data and write to CSV
            row_data.extend(landmark_data)
            row_data.extend(ball_data)
            row_data.extend(racquet_data)
            row_data.extend(calculated_metrics)
            csv_writer.writerow(row_data)

            # Write the frame to the output video
            out.write(frame)

        # Release resources for this video
        cap.release()
        out.release()

        print(f"\nProcessed video {video_file} saved to: {output_video_path}")
        print(f"Analysis data saved to: {csv_filename}")
        print(f"Maximum Ball Height: {max_ball_height_cm:.2f} cm")
        print(f"Maximum Wrist Arc Curvature: {max_arc_curvature:.2f} deg")
        print(f"Maximum Angular Velocity: {max_angular_velocity:.2f} deg/s")
        if service_motion_time is not None:
            print(f"Service Motion Time: {service_motion_time:.3f} s")

print("\nAll videos processed successfully!")


0: 384x640 1 ball, 1707.8ms
Speed: 0.0ms preprocess, 1707.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 racquet , 791.7ms
Speed: 0.0ms preprocess, 791.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Serve motion detected!

0: 384x640 (no detections), 1694.3ms
Speed: 0.0ms preprocess, 1694.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 racquet , 799.1ms
Speed: 6.7ms preprocess, 799.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1687.9ms
Speed: 6.6ms preprocess, 1687.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 racquet , 796.2ms
Speed: 3.9ms preprocess, 796.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1673.6ms
Speed: 0.0ms preprocess, 1673.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 racquet , 770.6ms
Speed: 6.6ms pre